# Import Packages

In [1]:
import glob
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import networkx as nx
from scipy.stats import kendalltau
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gensim


# Processing Data

In [ ]:
txts = glob.glob("C:/Users/yiwei/Downloads/wiki_data/*.txt")
titles = []
texts = []
for val in tqdm(txts):
    with open(val) as f:
        text = f.read()
        title = text[text.find('>') + 1:text.find('<', 2)]
        texts.append(text)
        titles.append(title)

100%|██████████| 40001/40001 [01:12<00:00, 552.31it/s]  


In [27]:
corpus = pd.DataFrame({'title': titles, 'text': texts})

In [36]:
corpus.head()

,title,text
0,April,<title>April</title><text>{{monththisyear|4}} ...
1,August,<title>August</title><text>{{monththisyear|8}}...
2,Andouille,<title>Andouille</title><text>[[File:Andouille...
3,Calculus,<title>Calculus</title><text>{{More citations ...
4,Liter,<title>Liter</title><text>#REDIRECT [[Litre]] ...


In [52]:
to_drop = []
for val in range(len(corpus['title'])):
    try:
        if 'template' in corpus['title'][val].lower() or 'category' in corpus['title'][val].lower():
            to_drop.append(val)
    except:
        pass

In [55]:
corpus.drop(to_drop, inplace=True)

In [56]:
corpus.to_csv('wiki_data.csv', index=False)

In [2]:
corpus = pd.read_csv('wiki_data.csv')

In [3]:
# because dataset is too big to handle with our resources (limitations for semantic graph), we subsample the data
subset_corpus = corpus.sample(frac=0.25, random_state=42)
subset_corpus.reset_index(drop=True, inplace=True)

In [59]:
subset_corpus.to_csv('subset_wiki_data.csv', index=False)

In [2]:
subset_corpus = pd.read_csv('subset_wiki_data.csv')

## Generating Semantic Graph from Dataset

In [3]:
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [4]:
# removing punctuation and stop words
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')
to_tokenize = []
for article in tqdm(subset_corpus['text']):
    cleaned_text = cleanhtml(article)
    tokens = tokenizer.tokenize(cleaned_text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    to_tokenize.append(filtered_tokens)

100%|██████████| 7908/7908 [00:00<00:00, 9549.12it/s]


In [5]:
idx_title_mapping = {idx: str(subset_corpus['title'][idx]).lower() for idx in range(len(to_tokenize))}

In [6]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [7]:
def get_text_embedding(tokens, model):
    word_vectors = []
    for word in tokens:
        if word in model.key_to_index: 
            word_vectors.append(model[word])
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

text_embeddings = np.array([get_text_embedding(tokens, model) for tokens in tqdm(to_tokenize)])

cos_sim_matrix = cosine_similarity(text_embeddings)

100%|██████████| 7908/7908 [00:03<00:00, 2346.39it/s]


In [8]:
SG = nx.Graph()

threshold = 0.1

for i in tqdm(range(len(to_tokenize))):
    SG.add_node(i)
    for j in range(i+1, len(to_tokenize)):
        similarity = cos_sim_matrix[i, j]
        if abs(similarity) >= threshold:
            SG.add_edge(i, j, weight=similarity)


100%|██████████| 7908/7908 [00:40<00:00, 193.97it/s] 


## Generating PageRank Graph from Dataset

In [8]:
PRG = nx.DiGraph()
pattern = r'\[\[(.*?)\]\]'
titles = subset_corpus['title'].str.lower()
PRG.add_nodes_from(titles)
for i in tqdm(range(len(subset_corpus['title']))):
    txt = subset_corpus['text'][i]
    references = re.findall(pattern, txt)
    for reference in references:
        if len(titles[titles == reference]) > 0:
            PRG.add_edge(titles[i], reference.lower())

100%|██████████| 7908/7908 [02:05<00:00, 63.01it/s] 


# Using Semantic Graph for PageRank

In [405]:
pr_semantic = nx.pagerank(SG, weight='weight')
pr_semantic = {idx_title_mapping[idx]: pr_semantic[idx] for idx in pr_semantic}

# Using Normal Dataset for PageRank

In [406]:
pr = nx.pagerank(PRG)

# Weighted Semantic Graph vs Normal Dataset for PageRank 

In [9]:
def sort_dict(input):
    keys = list(input.keys())
    values = list(input.values())
    sorted_value_idx = np.argsort(values)
    return [keys[i] for i in sorted_value_idx[::-1]]

In [408]:
pr_sorted = sort_dict(pr)
pr_semantic_sorted = sort_dict(pr_semantic)

In [409]:
kendalltau(pr_sorted, pr_semantic_sorted)

SignificanceResult(statistic=0.008867568973233186, pvalue=0.23795417239008265)

In [410]:
pr_semantic_sorted[:10]

['aleister crowley',
 'history of graphic design',
 'romania',
 'chuck berry',
 'gregor mendel',
 'port arthur, tasmania',
 'bristol',
 '1985',
 'human sacrifice',
 'joseph beuys']

In [411]:
pr_sorted[:10]

['government',
 'language',
 'species',
 'actor',
 'money',
 'circle',
 'currency',
 '2007',
 'air',
 'computer']

# Unweighted Semantic Graph

In [412]:
SGU = nx.DiGraph()

threshold = 0.8

for i in tqdm(range(len(to_tokenize))):
    SGU.add_node(i)
    for j in range(i+1, len(to_tokenize)):
        similarity = cos_sim_matrix[i, j]
        if similarity >= threshold:
            SGU.add_edge(i, j)

  0%|          | 0/7908 [00:00<?, ?it/s]

100%|██████████| 7908/7908 [00:07<00:00, 1046.52it/s]


In [413]:
pr_semantic_unweighted = nx.pagerank(SGU, weight='weight')
pr_semantic_unweighted = {idx_title_mapping[idx]: pr_semantic_unweighted[idx] for idx in pr_semantic_unweighted}

In [414]:
pr_semantic_unweighted_sorted = sort_dict(pr_semantic_unweighted)

In [415]:
kendalltau(pr_sorted, pr_semantic_unweighted_sorted)

SignificanceResult(statistic=-0.00029786599255738196, pvalue=0.968379604691019)

In [416]:
kendalltau(pr_semantic_sorted, pr_semantic_unweighted_sorted)

SignificanceResult(statistic=0.001246908652547464, pvalue=0.8682030881727664)

In [417]:
pr_semantic_unweighted_sorted[:10]

['the new york times',
 '2007',
 'isle of man',
 'joseph beuys',
 'bad zurzach',
 'silkworm',
 'binomial nomenclature',
 'hedgehog',
 'janice raymond',
 'new brunswick']

# Combining Weights

In [12]:
# creating graph function

def create_graph(weighted = True, alpha = 0.1, user_input = None, cos_sim_matrix = cos_sim_matrix):
    # alpha is the semantic value score, 1 - alpha is link score
    OUT_GRAPH = nx.Graph()
    matrix_len = cos_sim_matrix.shape[0]
    if user_input:
        tokenized_user_input = tokenizer.tokenize(cleanhtml(user_input))
        filtered_user_input = [word for word in tokenized_user_input if word not in stop_words]
        user_embedding = get_text_embedding(filtered_user_input, model)
        cos_sim_matrix = cosine_similarity(np.vstack([text_embeddings, user_embedding]))
        matrix_len = cos_sim_matrix.shape[0]
        idx_title_mapping[matrix_len - 1] = f'user input: {user_input}'
    for i in tqdm(range(matrix_len)):
        OUT_GRAPH.add_node(i)
        for j in range(i+1, matrix_len):
            similarity = cos_sim_matrix[i, j] 
            link = int((idx_title_mapping[i], idx_title_mapping[j]) in PRG.edges) if j < matrix_len - 1 and i < matrix_len - 1 else 0
            if weighted:
                score = similarity * alpha + (1 - alpha) * link if abs(similarity) >= 0.1 else (1 - alpha) * link
            else:
                if user_input:
                    score = alpha + (1 - alpha) * link if similarity >= 0.1 else 0
                else:
                    score = alpha + (1 - alpha) * link if similarity >= 0.8 else (1 - alpha) * link
            if score:
                OUT_GRAPH.add_edge(i, j, weight=score)
    return OUT_GRAPH


In [10]:
alphas = [0.075, 0.1, 0.125]

In [13]:
def evaluate_top_ten(OUT_GRAPH, user_input = None):
    evaluated = nx.pagerank(OUT_GRAPH, weight='weight')
    evaluated = {idx_title_mapping[idx]: evaluated[idx] for idx in evaluated}
    evaluated_sorted = sort_dict(evaluated)
    return evaluated_sorted[:10], evaluated

## Weighted Graph (combine using linear combination)

In [14]:
weighted_t10 = {}
weighted_rankings = {}
for alpha in alphas:
    print(f'Starting alpha value {alpha}...')
    OUT_GRAPH = create_graph(True, alpha)
    top_ten, ranking_vals = evaluate_top_ten(OUT_GRAPH)
    weighted_t10[alpha] = top_ten
    weighted_rankings[alpha] = ranking_vals


Starting alpha value 0.075...


100%|██████████| 7908/7908 [00:51<00:00, 154.93it/s] 


Starting alpha value 0.1...


100%|██████████| 7908/7908 [00:52<00:00, 150.10it/s] 


Starting alpha value 0.125...


100%|██████████| 7908/7908 [00:53<00:00, 147.51it/s] 


In [15]:
for alpha in weighted_t10:
    print(f'For alpha value {alpha}:')
    print(f'Top ten articles are {", ".join(weighted_t10[alpha])}')

For alpha value 0.075:
Top ten articles are 2007, 2002, wikipedia:list of common misspellings, 2001, 1990, 1988, 1986, 1949, 1927, 1916
For alpha value 0.1:
Top ten articles are 2007, 2002, 2001, wikipedia:list of common misspellings, 1990, 1988, 1986, 1949, 1993, 1927
For alpha value 0.125:
Top ten articles are 2007, 2002, 2001, 1990, wikipedia:list of common misspellings, 1988, 1993, 1986, 1949, 1916


## Unweighted Graph (combine using linear combination)

In [16]:
unweighted_t10 = {}
unweighted_rankings = {}
for alpha in alphas:
    print(f'Starting alpha value {alpha}...')
    OUT_GRAPH = create_graph(False, alpha)
    top_ten, ranking_vals = evaluate_top_ten(OUT_GRAPH)
    unweighted_t10[alpha] = top_ten
    unweighted_rankings[alpha] = ranking_vals


Starting alpha value 0.075...


100%|██████████| 7908/7908 [00:17<00:00, 455.56it/s] 


Starting alpha value 0.1...


100%|██████████| 7908/7908 [00:14<00:00, 528.63it/s] 


Starting alpha value 0.125...


100%|██████████| 7908/7908 [00:14<00:00, 536.99it/s] 


In [17]:
for alpha in unweighted_t10:
    print(f'For alpha value {alpha}:')
    print(f'Top ten articles are {", ".join(unweighted_t10[alpha])}')

For alpha value 0.075:
Top ten articles are wikipedia:list of common misspellings, wikipedia:basic english ordered wordlist, 2007, list of years, 2002, language, 1988, 2001, 1990, 1986
For alpha value 0.1:
Top ten articles are wikipedia:list of common misspellings, wikipedia:basic english ordered wordlist, 2007, list of years, 2002, 1988, 2001, language, 1990, 1986
For alpha value 0.125:
Top ten articles are wikipedia:list of common misspellings, wikipedia:basic english ordered wordlist, 2007, 2002, list of years, 1988, 2001, 1990, language, curitiba


# Using Combined Graph for User Input

In [10]:
def sorted_results(graph):
    scores = [val for val in sorted(graph.edges(data=True),key= lambda x: x[2]['weight'],reverse=True) if len(to_tokenize) in val]
    search_results = [idx_title_mapping[val[0]] for val in scores]
    return search_results, scores

## Combined Graph Weighted

In [19]:
USER_WEIGHTED_GRAPH = create_graph(True, 0.125, 'funny movies')
user_search_results, user_scores = sorted_results(USER_WEIGHTED_GRAPH)

100%|██████████| 7909/7909 [00:51<00:00, 154.68it/s] 


In [20]:
user_search_results[:10]

['parody',
 '2007 in film',
 'trailer (movie)',
 'christopher walken',
 'high school musical',
 'philadelphia (movie)',
 'sunrise (movie)',
 'tron',
 'rounders (movie)',
 'office space']

### Combined Graph Unweighted

In [14]:
USER_UNWEIGHTED_GRAPH = create_graph(False, 0.125, 'funny movies')
user_search_results, user_scores = sorted_results(USER_UNWEIGHTED_GRAPH)


100%|██████████| 7909/7909 [00:46<00:00, 170.37it/s] 


In [15]:
user_search_results[:10]

['knight of the british empire',
 'miss world',
 'world war',
 'tripoli, greece',
 'jungle',
 'pathogen',
 'message (computer science)',
 'adam elliot',
 'bush',
 'edinburgh castle']